In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
import pickle
import math
from scipy.stats import pearsonr, spearmanr, kendalltau, shapiro, pointbiserialr
from sklearn.linear_model import RandomizedLogisticRegression as RLR

import os,re,sys
from flask import Flask, abort, jsonify, request, session


In [2]:
def createfeature(df):
    
    df['Average Donation per Month'] = (df['Total Volume Donated (c.c.)']/df['Months since First Donation'])
    df['Waiting Time'] = ((df['Months since First Donation'] - df['Months since Last Donation'])/df['Number of Donations'])
    #df['Donated in the past 3-6 months'] = ((df['Months since Last Donation'] >= 3) &(df['Months since Last Donation'] <= 6))
    #df['Frequent Donor'] = (df['Number of Donations'] >= 5)
    df['monthratio']=df['Months since Last Donation']/df['Months since First Donation']
    #df['monthratio']=df['Months since Last Donation']/df['Months since First Donation']
   
       
    df['Number of Donations sqrt']=np.sqrt(df['Number of Donations'].astype(float))
    df['Average Donation per Month sqrt']=np.sqrt(df['Average Donation per Month'])
    df['Months since First Donation sqrt']=np.sqrt(df['Months since First Donation'])
    df['Waiting Time sqrt']=np.sqrt(df['Waiting Time'])
    df['Months since Last Donation sqrt']=np.sqrt(df['Months since Last Donation'])

    df['Months since Last Donation mul']=df['Months since Last Donation']*df['Months since Last Donation']
    df['Number of Donations mul']=df['Number of Donations']*df['Number of Donations']
    df['Average Donation per Month mul']=df['Average Donation per Month']*df['Average Donation per Month']
    df['Waiting Time mul']=df['Waiting Time']*df['Waiting Time']
    df['Months since First Donation mul']=df['Months since First Donation']*df['Months since First Donation']
    
 
    means=df.mean(axis=1)
    df['mean']=means
    MAX=df.max(axis=1)
    df['max']=MAX
    MIN=df.min(axis=1)
    df['min']=MIN
    STD=df.std(axis=1)
    df['std']=STD
    MEDIAN=df.median(axis=1)
    df['median']=MEDIAN
    kurtosis=df.kurtosis(axis=1)
    df['kurtosis']=kurtosis
    skew=df.skew(axis=1)
    df['skew']=skew
    var=df.var(axis=1)
    df['var']=var

    return df

In [3]:
blood_model = pickle.load(open("blood_model.pickle","rb" ) )

In [4]:
app = Flask(__name__)

In [ ]:
@app.route('/blood_mode', methods = ['POST'])
def blood_mode():
    
    try:
        
        data = request.get_json(force = True)
        
        filepath = data['filepath']
        #print(filepath)
        test=pd.read_csv(filepath)
        #print(test.head(5))
        
        test=createfeature(test)
        
        test=createfeature(test)
        test = test.drop(["Total Volume Donated (c.c.)"],axis=1)
        
        y_pred = blood_model.predict(test)
        pred = y_pred.tolist()
        
        
    except Exception as e:
        return jsonify ({'Error':str(e),'status':'False'})
        
    else:
        return jsonify({'prediction':pred})

In [ ]:
app.run("0.0.0.0", port = 7001, debug = True, use_reloader = False,threaded=True)

 * Running on http://0.0.0.0:7001/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Sep/2018 12:56:16] "POST /blood_mode HTTP/1.1" 200 -


In [ ]:
test=pd.read_csv('test.csv')
#print(test.head(5))

test=createfeature(test)

test=createfeature(test)
test = test.drop(["Total Volume Donated (c.c.)"],axis=1)
print(test.shape)


In [ ]:
y_pred = blood_model.predict(test)

In [ ]:
y_pred